<a href="https://colab.research.google.com/github/naikamal/HandsOnPytorch/blob/main/DeepLearning_Projects/RNNs/next_letter_prediction_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import Libraries

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Step 2: Build Character Vocabulary


In [6]:
names=['naikamalshah','raheemshah','taimurkhan','dayankhan']

all_chars=sorted(set("".join(names)))
char_to_idx={char:i for i,char in enumerate(all_chars)} # maps 'a' → 0, 'h' → 1, etc.
idx_to_char={i:char for char,i in char_to_idx.items()}
vocab_size=len(all_chars)
print(f"All unique characters: {all_chars}")
# print(f"char to idx: {char_to_idx}")
# print(f"idx to char: {idx_to_char}")
print(f"Vocablary size: {vocab_size}")

All unique characters: ['a', 'd', 'e', 'h', 'i', 'k', 'l', 'm', 'n', 'r', 's', 't', 'u', 'y']
Vocablary size: 14


# Step 3: Prepare Dataset

In [7]:
def make_pairs(name):
  inputs,targets=[],[]
  for i in range(len(name)-1):
    inputs.append(char_to_idx[name[i]])
    targets.append(char_to_idx[name[i+1]])

  return inputs,targets


# Step 4: Define the RNN Model

In [12]:
class CharRNN(nn.Module):
  def __init__(self,vocab_size,hidden_size):
    super().__init__()
    self.embed=nn.Embedding(vocab_size,hidden_size)
    self.rnn=nn.RNN(hidden_size,hidden_size,batch_first=True)
    self.fc=nn.Linear(hidden_size,vocab_size)


#  Forward Function
  def forward(self,x,hidden):
    x=self.embed(x)
    out,hidden=self.rnn(x,hidden)
    out=self.fc(out)
    return out,hidden

# Flow of data:
# [char_indices] → embed → rnn → linear → predictions



# 5. Model Setup

In [13]:
hidden_size=32
model=CharRNN(vocab_size,hidden_size)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)




# Step 5: Train the RNN

In [15]:
for epoch in range(200):
  name=random.choice(names)
  inputs_idx,targets_idx=make_pairs(name)

  input_tensor=torch.tensor([inputs_idx],dtype=torch.long)
  target_tensor=torch.tensor([targets_idx],dtype=torch.long)

  optimizer.zero_grad()
  hidden=torch.zeros(1,1,hidden_size) #num_layers=1, batch=1

  output,new_hidden=model(input_tensor,hidden)
  loss=criterion(output.squeeze(0),target_tensor.squeeze(0))

  loss.backward()
  optimizer.step()

  if epoch%20==0:
    print(f"Epoch:{epoch}, Loss:{loss.item():.4f}")






Epoch:0, Loss:2.6882
Epoch:20, Loss:0.7618
Epoch:40, Loss:0.2478
Epoch:60, Loss:0.1036
Epoch:80, Loss:0.0724
Epoch:100, Loss:0.0228
Epoch:120, Loss:0.0187
Epoch:140, Loss:0.0167
Epoch:160, Loss:0.0107
Epoch:180, Loss:0.0114


# 6. Predicting Next Character

In [18]:
def predict_next(name_seed, model,hidden_size):
  for ch in name_seed:
    if ch not in char_to_idx:
      raise ValueError(f"character {ch} is not in vocablary!")

  input_idx=[char_to_idx[ch] for ch in name_seed]
  input_tensor=torch.tensor([input_idx],dtype=torch.long)
  hidden=torch.zeros(1,1,hidden_size)

  output,new_hidden=model(input_tensor,hidden)
  last_logits = output[0, -1]  # output of last time step
  pred_idx=torch.argmax(last_logits).item()

  return idx_to_char[pred_idx]

print(predict_next("kha",model,hidden_size))

n
